<a href="https://colab.research.google.com/github/snufkin92/nlp_bert/blob/master/section_02/03_simple_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# シンプルなBERTの実装
訓練済みのモデルを使用し、文章の一部の予測、及び2つの文章が連続しているかどうかの判定を行います。

## ライブラリのインストール
PyTorch-Transformers、および必要なライブラリのインストールを行います。

In [1]:
!pip install folium==0.2.1
!pip install urllib3==1.25.11
!pip install pytorch-transformers==1.2.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.0/70.0 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for folium: filename=folium-0.2.1-py3-none-any.whl size=79793 sha256=a9cf191756f432ee70feb5621932d8a34e5e7823921de1727881c2e91943f1e1
  Stored in directory: /root/.cache/pip/wheels/00/0c/07/d7792a5444d5bb074361ac27da53cee9d5cce59a07fe9da5dd
Successfully built folium
  Attempting uninstall: folium
    Found existing installation: folium 0.17.0
    Uninstalling folium-0.17.0:
      Successfully uninstalled folium-0.17.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.17.6 requires folium>=0.9.1, but you have folium 0.2.1 which is incompatible.
geemap 0.33.1 requires folium>=0.13.0, but you have folium 0.2.1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 2.4 MB/s eta 0:00:00
   ━━━

## 文章の一部の予測
文章における一部の単語をMASKし、それをBERTのモデルを使って予測します。

In [2]:
import torch
from pytorch_transformers import BertForMaskedLM
from pytorch_transformers import BertTokenizer

# 「CLS]、「SEP] はBERTにとって特別な意味(文章の開始と終了）を持つ
text = "[CLS] I played baseball with my friends at school yesterday [SEP]"
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
words = tokenizer.tokenize(text)
print(words)

100%|██████████| 231508/231508 [00:00<00:00, 1292645.92B/s]


['[CLS]', 'i', 'played', 'baseball', 'with', 'my', 'friends', 'at', 'school', 'yesterday', '[SEP]']


文章の一部をMASKします。

In [3]:
msk_idx = 3

# 3番目のインデックス（baseball）をマスク
words[msk_idx] = "[MASK]"  # 単語を[MASK]に置き換える
print(words)

['[CLS]', 'i', 'played', '[MASK]', 'with', 'my', 'friends', 'at', 'school', 'yesterday', '[SEP]']


単語を対応するインデックスに変換します。

In [4]:
# 事前学習済のBERT単語辞書の対応するインデックスに変換
# もし単語辞書に存在しない単語 (Out-of-Vocabulary, OOV) があった場合、
# BERT トークナイザーはそれをサブワードに分割するか、特別なトークン [UNK] (Unknown) に置き換える。
# 例：unbelievableが登録されていなかった場合、
## "un#と "#believ" と "##able" に分割する可能性がある
# 　"believ" と "able" は単語辞書の中に存在し、単語の途中に現れることを示すために、
# 接頭語として "##" が付けられる。
# ## は BERT ではトークンの位置情報を示す特別な記号としての意味を持つ
# ただし "believ"は単語辞書に存在しないため "believ"のインデックスに
# サブワードである事を示す情報が付加される（Position Embeddingで解決）
word_ids = tokenizer.convert_tokens_to_ids(words)  # 単語をインデックスに変換
print("### type(word_ids) = ", type(word_ids))
print(word_ids)

word_tensor = torch.tensor([word_ids])  # テンソルに変換
print("### type(word_tensor) = ", type(word_tensor))
print(word_tensor)

### type(word_ids) =  <class 'list'>
[101, 1045, 2209, 103, 2007, 2026, 2814, 2012, 2082, 7483, 102]
### type(word_tensor) =  <class 'torch.Tensor'>
tensor([[ 101, 1045, 2209,  103, 2007, 2026, 2814, 2012, 2082, 7483,  102]])


In [7]:
len(word_ids)

11

BERTのモデルを使って予測を行います。

In [15]:
msk_model = BertForMaskedLM.from_pretrained("bert-base-uncased")

# GPUだと数秒、CPUだと絶望的に遅い
msk_model.cuda()  # GPU対応
msk_model.eval() # 学習は行わず予測モード

x = word_tensor.cuda()  # GPU対応
# x = word_tensor # CPU用のテンソル
y = msk_model(x)  # 予測
print("### type(y) = ", type(y))
result = y[0]

# len(word_ids) = 11
# torch.Size([1, 11, 30522]) = [バッチサイズ、単語数、単語数]
print(result.size())  # 結果の形状

# "_"はTop5の単語のスコア（対数尤度）
_, top5_indices = torch.topk(result[0][msk_idx], k=5)  # 最も大きい5つの値
print("### _ = ", _)
print("### max_ids = ", max_ids)

# [MASK]に入る部分の単語
result_words = tokenizer.convert_ids_to_tokens(top5_indices.tolist())  # インデックスを単語に変換
print(result_words)

# result_wordsに対する確率を出力
prob = torch.nn.functional.softmax(result[0][msk_idx][top5_indices], dim=0)
print(prob)

### type(y) =  <class 'tuple'>
torch.Size([1, 11, 30522])
### _ =  tensor([10.5885, 10.5762,  9.9564,  9.8097,  9.1332], device='cuda:0',
       grad_fn=<TopkBackward0>)
### max_ids =  tensor([3455, 2374, 4715, 3598, 5093], device='cuda:0')
['basketball', 'football', 'soccer', 'baseball', 'tennis']
tensor([0.3114, 0.3076, 0.1655, 0.1429, 0.0727], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)


In [10]:
print("### result[0] = ", result[0])

### result[0] =  tensor([[ -6.6873,  -6.6405,  -6.6409,  ...,  -6.0201,  -5.8183,  -3.9777],
        [ -9.5150,  -9.3415,  -9.3818,  ...,  -8.4236,  -8.4428,  -5.3152],
        [-10.0568, -10.1768, -10.2753,  ...,  -8.5044,  -8.6216,  -5.3011],
        ...,
        [-13.6662, -14.2769, -13.8572,  ..., -12.8681, -11.8016, -11.4662],
        [ -9.2015,  -8.9383,  -9.3056,  ...,  -7.7869,  -9.2608,  -3.0500],
        [-13.1242, -12.9603, -12.7900,  ...,  -9.9769, -10.1773, -10.8939]],
       device='cuda:0', grad_fn=<SelectBackward0>)


In [11]:
# 勿論、out of bounds
print("### result[1] = ", result[1])

IndexError: index 1 is out of bounds for dimension 0 with size 1

In [12]:
# 入力各単語の確率（30522語の中での確率）
print("### result[0].shpae] = ", result[0].shape)


### result[0].shpae] =  torch.Size([11, 30522])


## 文章が連続しているかどうかの判定
BERTのモデルを使って、2つの文章が連続しているかどうかの判定を行います。  
以下の関数`show_continuity`では、2つの文章の連続性を判定し、表示します。

In [17]:
from pytorch_transformers import BertForNextSentencePrediction

def show_continuity(text, seg_ids):
    """
    text: 2つの文章を"[CLS]"と"[SEP]"トークンで区切って結合した文字列。
        例: "[CLS] 今日はいい天気です。[SEP] 公園でピクニックをします。[SEP]"

    seg_ids: 各トークンがどちらの文章に属するかを示す数値のリスト。
             最初の文章のトークンには0、2つ目の文章のトークンには1が割り当てられる。
        例: [0, 0, 0, 0, 0, 1, 1, 1, 1, 1]

    """
    words = tokenizer.tokenize(text)
    word_ids = tokenizer.convert_tokens_to_ids(words)  # 単語をインデックスに変換
    word_tensor = torch.tensor([word_ids])  # テンソルに変換

    seg_tensor = torch.tensor([seg_ids])

    nsp_model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')
    nsp_model.cuda()  # GPU対応
    nsp_model.eval()

    x = word_tensor.cuda()  # GPU対応
    s = seg_tensor.cuda()  # GPU対応

    y = nsp_model(x, s)  # 予測
    result = torch.softmax(y[0], dim=1)
    print(result)  # Softmaxで確率に
    print(str(result[0][0].item()*100) + "%の確率で連続しています。")

`show_continuity`関数に、自然につながる2つの文章を与えます。

In [11]:
text = "[CLS] What is baseball ? [SEP] It is a game of hitting the ball with the bat [SEP]"
seg_ids = [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1 ,1, 1]  # 0:前の文章の単語、1:後の文章の単語
show_continuity(text, seg_ids)

tensor([[1.0000e+00, 4.5869e-06]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
99.9995470046997%の確率で連続しています。


`show_continuity`関数に、自然につながらない2つの文章を与えます。

In [12]:
text = "[CLS] What is baseball ? [SEP] This food is made with flour and milk [SEP]"
seg_ids = [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]  # 0:前の文章の単語、1:後の文章の単語
show_continuity(text, seg_ids)

tensor([[9.5296e-06, 9.9999e-01]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
0.0009529620001558214%の確率で連続しています。


お遊び

In [27]:
# text = "[CLS] I love you [SEP] Me too [SEP]" # 99.99316930770874%
# text = "[CLS] I love you [SEP] Kick ass [SEP]" # 7.2146937251091%
text = "[CLS] I love you [SEP] Fuck you [SEP]"
seg_ids = [0, 0, 0, 0, 0, 1, 1, 1]  # 0:前の文章の単語、1:後の文章の単語
show_continuity(text, seg_ids)

tensor([[0.9984, 0.0016]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
99.84017014503479%の確率で連続しています。
